### 1. Initialize Habitat Environment

In [1]:
import logging

# Habitat 내부 로그 레벨 조정 (Warning 숨기기)
logging.getLogger("habitat").setLevel(logging.ERROR)
logging.getLogger("magnum").setLevel(logging.ERROR)
logging.getLogger("assets").setLevel(logging.ERROR)

In [2]:
import git, os
repo = git.Repo(".", search_parent_directories=True)
dir_path = repo.working_tree_dir
data_path = os.path.join(dir_path, "data")
os.chdir(dir_path)

oracle pick and place를 사용하기 위해, habitat_llm의 actions를 사용합니다.  
아래처럼 import를 수행하면, registry에 등록되기 때문에, yaml로부터 불러올 수 있습니다. (test.yaml 참고)   
oracle nav의 경우도 habitat_llm에 존재하지만, _EnvironmentInterface_를 활용하기 때문에, habitat_lab의 oracle_nav를 사용합니다.  
habitat_lab의 oracle_nav는 pddl에 기반해서 움직이기 때문에, PDDLTask에만 사용 가능합니다.

In [3]:
import habitat
from habitat.datasets.rearrange.rearrange_dataset import RearrangeDatasetV0
from habitat_llm.agent.env.actions import OraclePickAction, OraclePlaceAction

# conf = habitat.get_config("workspace/test.yaml")
conf = habitat.get_config("/partnr-planner/third_party/habitat-lab/habitat-lab/habitat/config/benchmark/rearrange/hab3_bench/test.yaml")

dataset = RearrangeDatasetV0(
                config=conf.habitat.dataset
            )
env = habitat.registry.get_env("GymHabitatEnv")(
                config=conf, dataset=dataset
            )

PluginManager::Manager: duplicate static plugin StbImageImporter, ignoring
PluginManager::Manager: duplicate static plugin GltfImporter, ignoring
PluginManager::Manager: duplicate static plugin BasisImporter, ignoring
PluginManager::Manager: duplicate static plugin AssimpImporter, ignoring
PluginManager::Manager: duplicate static plugin AnySceneImporter, ignoring
PluginManager::Manager: duplicate static plugin AnyImageImporter, ignoring
Neither `ifconfig` (`ifconfig -a`) nor `ip` (`ip address show`) commands are available, listing network interfaces is likely to fail


/partnr-planner/third_party/habitat-lab/habitat-lab/habitat/config
/partnr-planner/third_party/habitat-lab/habitat-lab/habitat/config/benchmark/rearrange/hab3_bench/test.yaml
/partnr-planner/third_party/habitat-lab/habitat-lab/habitat/config/benchmark/rearrange/hab3_bench/test.yaml


2025-02-12 08:37:52,552 initializing sim RearrangeSim-v0
PluginManager::Manager: duplicate static plugin StbImageImporter, ignoring
PluginManager::Manager: duplicate static plugin GltfImporter, ignoring
PluginManager::Manager: duplicate static plugin BasisImporter, ignoring
PluginManager::Manager: duplicate static plugin AssimpImporter, ignoring
PluginManager::Manager: duplicate static plugin AnySceneImporter, ignoring
PluginManager::Manager: duplicate static plugin AnyImageImporter, ignoring
[08:37:55:276555]:[Warning]:[Metadata] SceneDatasetAttributes.cpp(107)::addNewSceneInstanceToDataset : Dataset : 'hssd-hab-uncluttered' : Lighting Layout Attributes 'data/scene_datasets/hssd-hab/scenes-uncluttered/105515448_173104512.scene_instance.json' specified in Scene Attributes but does not exist in dataset, so creating default.
[08:37:55:403471]:[Warning]:[Sim] Simulator.cpp(599)::instanceStageForSceneAttributes : The active scene does not contain semantic annotations : activeSemanticSceneI

Renderer: NVIDIA GeForce RTX 4090/PCIe/SSE2 by NVIDIA Corporation
OpenGL version: 4.6.0 NVIDIA 535.171.04
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_separate_shader_objects
    GL_ARB_robustness
    GL_ARB_texture_storage
    GL_ARB_texture_view
    GL_ARB_framebuffer_no_attachments
    GL_ARB_invalidate_subdata
    GL_ARB_texture_storage_multisample
    GL_ARB_multi_bind
    GL_ARB_direct_state_access
    GL_ARB_get_texture_sub_image
    GL_ARB_texture_filter_anisotropic
    GL_KHR_debug
    GL_KHR_parallel_shader_compile
    GL_NV_depth_buffer_float
Using driver workarounds:
    no-forward-compatible-core-context
    nv-egl-incorrect-gl11-function-pointers
    no-layout-qualifiers-on-old-glsl
    nv-zero-context-profile-mask
    nv-implementation-color-read-format-dsa-broken
    nv-cubemap-inconsistent-compressed-image-size
    nv-cubemap-broken-full-compressed-image-query
    nv-compressed-block-size-in-bits


/opt/conda/envs/habitat/lib/python3.9/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


이렇게 직접 가져와서 사용도 가능한데, 불필요해보임

In [4]:
from habitat.core.registry import registry

action_class = registry.get_task_action("OraclePickAction")
action_instance = action_class(config=conf, sim=env.env.env._env.sim, name="oracle_pick")

### 2. Extract episode information (원래는 environment interface에서 활용)

환경을 만들었으니, episode로부터 task 수행에 필요한 것들의 정보를 추출해야 함.  
이 역할을 habitat_llm에선 EnvironmentInterface가 수행하는데, 여기선 간단하게 필요한 것만 추출해보겠음.   

In [5]:
def make_setting(env):
    habitat_env = env.env.env._env
    episode = habitat_env.current_episode
    task = habitat_env._task
    sim = habitat_env._sim
    return habitat_env, episode, task, sim

habitat_env, episode, task, sim = make_setting(env)

print(f"current_episode_id: {episode.episode_id}")
print(f"scene_id: {episode.scene_id}")
print(f"start_position (agent의 시작 위치): {episode.start_position}")
print(f"start_rotation (agent의 시작 방향): {episode.start_rotation}\n")

# print(f"target_receptacles (target이 위치하고 있는 receptacle):\n {episode.target_receptacles}\n")
# print(f"targets (targets의 위치):\n {episode.targets}\n")

current_episode_id: 23775
scene_id: data/scene_datasets/hssd-hab/scenes-uncluttered/105515448_173104512.scene_instance.json
start_position (agent의 시작 위치): [0, 0, 0]
start_rotation (agent의 시작 방향): [0, 0, 0, 1]



In [6]:
habitat_env.reset()

[08:37:58:149687]:[Warning]:[Assets] ResourceManager.cpp(897)::createRenderAssetInstance : Instantiating render asset `data/robots/hab_spot_arm/urdf/../meshesColored/base.glb` with incompatible light setup, instance will not be correctly lit. For objects, please ensure 'requires lighting' is enabled in object config file.
[08:37:58:150828]:[Warning]:[Assets] ResourceManager.cpp(897)::createRenderAssetInstance : Instantiating render asset `data/robots/hab_spot_arm/urdf/../meshesColored/fl.hip.glb` with incompatible light setup, instance will not be correctly lit. For objects, please ensure 'requires lighting' is enabled in object config file.
[08:37:58:150859]:[Warning]:[Assets] ResourceManager.cpp(897)::createRenderAssetInstance : Instantiating render asset `data/robots/hab_spot_arm/urdf/../meshesColored/fl.uleg.glb` with incompatible light setup, instance will not be correctly lit. For objects, please ensure 'requires lighting' is enabled in object config file.
[08:37:58:150892]:[Warn

{'head_rgb': array([[[ 72,  60,  47],
         [ 72,  60,  47],
         [ 72,  60,  47],
         ...,
         [103,  94,  83],
         [103,  94,  83],
         [ 72,  60,  47]],
 
        [[ 72,  60,  47],
         [ 72,  60,  47],
         [ 72,  60,  47],
         ...,
         [103,  94,  83],
         [103,  94,  83],
         [ 72,  60,  47]],
 
        [[ 72,  60,  47],
         [ 72,  60,  47],
         [ 72,  60,  47],
         ...,
         [103,  94,  83],
         [103,  94,  83],
         [ 72,  61,  47]],
 
        ...,
 
        [[209, 200, 177],
         [209, 200, 177],
         [209, 200, 177],
         ...,
         [102,  93,  82],
         [102,  94,  82],
         [209, 200, 178]],
 
        [[209, 200, 177],
         [208, 199, 177],
         [208, 199, 177],
         ...,
         [102,  94,  82],
         [102,  94,  82],
         [209, 200, 178]],
 
        [[208, 199, 177],
         [208, 199, 177],
         [208, 199, 177],
         ...,
         [102,  

원래는 이런 식으로 episode iteration이 진행됩니다.  

In [7]:
def iterate_episodes(habitat_env):
    num_episodes = len(habitat_env.episodes)
    for i in range(num_episodes):
        print(habitat_env.current_episode)
        episode = habitat_env.current_episode
        task = habitat_env._task
        sim = habitat_env._sim
        # TODO: task 수행하기
        
        habitat_env.reset() # episode reset하면 다음 episode로 넘어가기
    
    return # experiment results 등을 반환하면 좋을듯

# iterate_episodes(habitat_env)
# 이런 식으로 할 수 있음

### 3. PDDL Parsing 수행

OracleNavAction을 사용하기 위해 PDDLTask를 골랐으므로, PDDL Parsing도 알아보겠습니다.  
일단, env.task.pddl_problem을 통해 확인할 수 있고, 그 외의 정보도 알 수 있습니다.

In [8]:
def parse_pddl(task):
    pddl_problem = task.pddl_problem
    init = pddl_problem.init
    goal = pddl_problem.goal
    target_ids = pddl_problem.sim_info.target_ids
    obj_ids = pddl_problem.sim_info.obj_ids
    return pddl_problem, init, goal, target_ids, obj_ids

pddl_problem, init, goal, target_ids, obj_ids = parse_pddl(task)

print(f"initial state:\n {init}\n")
print(f"goal state:\n {goal}\n")
print(f"target_ids:\n {target_ids}\n")
print(f"obj_ids:\n {obj_ids}\n")


initial state:
 []

goal state:
 (LogicalExprType.AND: [<Predicate: object_at [[obj-T:movable_entity_type, at_entity-T:static_obj_type]] [[any_targets|0-T:movable_entity_type, TARGET_any_targets|0-T:goal_entity_type]]>, <Predicate: object_at [[obj-T:movable_entity_type, at_entity-T:static_obj_type]] [[any_targets|1-T:movable_entity_type, TARGET_any_targets|1-T:goal_entity_type]]>, <Predicate: not_holding [[robot-T:robot_entity_type]] [[robot_0-T:robot_entity_type]]>]

target_ids:
 {'TARGET_any_targets|0': 1, 'TARGET_any_targets|1': 3}

obj_ids:
 {'002_master_chef_can_:0000': 0, '004_sugar_box_:0000': 1, 'any_targets|0': 1, '007_tuna_fish_can_:0000': 2, '009_gelatin_box_:0000': 3, 'any_targets|1': 3, '024_bowl_:0000': 4}



OracleNavAction의 경우는 pddl_ordered_entities를 활용합니다.  
즉, pddl 상에서 any_targets|X 등으로 표시된 것들을 모아놓은 것인데, 이 list에서의 index를 통해 물건을 찾고 Oracle_path를 계산해 navigation합니다.  
따라서, 0번째, 1번째의 target의 ordered_entities 상에서 몇번째 index인지 확인합니다.  

In [9]:
import numpy as np

def filter_target_id(obj_ids):
    result = {}
    for key, value in obj_ids.items():
        # "any_targets|"가 포함된 키만 처리
        if "any_targets|" in key:
            # "any_targets|X"에서 X를 추출
            target_index = key.split("|")[-1]
            # "X:Y" 형식으로 결과에 추가
            result[int(target_index)] = np.array([value])
    return result

target_ids = filter_target_id(obj_ids)

하지만, habitat_llm의 pick&place는 habitat-lab, pddl과 연관이 없습니다.  
habitat-lab의 grasp action은 팔을 움직이는 것 까지 포함되어 있어, 현재 단계에선 사용하지 않았습니다.  
habitat_llm의 pick&place는 각각 object의 sim 속 id와 target의 위치가 필요합니다.  
이는 sim으로 부터 가져와야 합니다.  
pddl_problem 상에선, 단지 predicates만 나열되어 있기 때문입니다.  
sim에서 target의 위치를 가져오려면 handle을 가져와야 합니다.  따라서, 아래 코드 작성

In [10]:
def get_handles(obj_ids):
    from collections import defaultdict

    value_to_keys = defaultdict(list)
    for key, value in obj_ids.items():
        value_to_keys[value].append(key)

    # 대표 키 선택 (any_targets| 제외하고 가장 먼저 등장하는 key 선택)
    reduced_dict = {}
    for value, keys in value_to_keys.items():
        if len(keys) > 1:  # 중복 등장하는 value만 처리
            # any_targets|가 포함되지 않은 key 찾기
            value = next((k.split("|")[-1] for k in keys if "any_targets|" in k), None)
            chosen_key = next((k for k in keys if "any_targets|" not in k), keys[0])
            reduced_dict[value] = chosen_key

    return reduced_dict

target_handles = get_handles(obj_ids)

In [11]:
print(target_ids)

{0: array([1]), 1: array([3])}


### 4. Objects, receptacles mapping

action에 제공하기 위해, 실제로 simulator 내에 어디 위치하는 지는 따로 찾아야 합니다.  
sim.get_rigid_object_manager(), sim.get_articulated_object_manager() 등을 활용합니다.

In [12]:
print(dir(episode))

['__annotations__', '__attrs_attrs__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_reset_shortest_path_cache_hook', '_shortest_path_cache', 'additional_obj_config_paths', 'ao_states', 'episode_id', 'goal_receptacles', 'info', 'markers', 'name_to_receptacle', 'rigid_objs', 'scene_dataset_config', 'scene_id', 'start_position', 'start_rotation', 'target_receptacles', 'targets']


현재 sim 속에 있는 object들의 위치는 어디인지 (episode.rigid_objs),  
우리가 옮겨야 할 target object는 무엇인지 확인합니다.

In [13]:
print(f"object_ids:\n {episode.rigid_objs}\n")
print(f"target_object_labels:\n {episode.info['object_labels']}\n")

object_ids:
 [['002_master_chef_can.object_config.json', [[0.81591, 0.0029, 0.57818, -12.99186], [-0.00304, 1.0, -0.00073, 0.51116], [-0.57818, -0.00116, 0.81591, -6.13115], [0.0, 0.0, 0.0, 1.0]]], ['004_sugar_box.object_config.json', [[0.2527, 0.01656, -0.9674, -7.4937], [-0.01419, 0.99981, 0.0134, 2.12802], [0.96744, 0.01034, 0.25288, -0.71232], [0.0, 0.0, 0.0, 1.0]]], ['007_tuna_fish_can.object_config.json', [[-0.01652, 0.09163, 0.99566, -1.88144], [0.01851, 0.99565, -0.09132, 0.876], [-0.99969, 0.01692, -0.01815, 2.67924], [0.0, 0.0, 0.0, 1.0]]], ['009_gelatin_box.object_config.json', [[0.88406, -0.004, 0.46735, -13.19167], [0.00575, 0.99998, -0.00232, 0.47981], [-0.46733, 0.00474, 0.88407, -7.28106], [0.0, 0.0, 0.0, 1.0]]], ['024_bowl.object_config.json', [[0.91974, 0.05226, -0.38903, -15.14722], [-0.0603, 0.99814, -0.00847, 0.54851], [0.38786, 0.03125, 0.92119, -7.4152], [0.0, 0.0, 0.0, 1.0]]]]

target_object_labels:
 {'004_sugar_box_:0000': 'any_targets|0', '009_gelatin_box_:000

target_object를 goal_receptacles로 옮겨야 하고, 그 위치는 targets에 적힌 위치와 동일합니다.  

In [14]:
print(f"goal_receptacles:\n {episode.goal_receptacles}\n")
print(f"targets:\n {episode.targets}\n")

goal_receptacles:
 [['c3bfe9d9969b21fac01b8bb7b9399a0047feef73_:0000', None], ['db256923d176b8dabf697f80067949282ac57326_:0000', None]]

targets:
 {'004_sugar_box_:0000': [[-0.87737, 0.0, -0.47981, -1.87741], [0.0, 1.0, 0.0, 0.58516], [0.47981, 0.0, -0.87737, 0.1998], [0.0, 0.0, 0.0, 1.0]], '009_gelatin_box_:0000': [[0.04764, 0.0, 0.99886, -19.20558], [0.0, 1.0, 0.0, 0.82784], [-0.99886, 0.0, 0.04764, -8.02568], [0.0, 0.0, 0.0, 1.0]]}



object의 위치를 파악하려면, rigid_object_manager로도 확인할 수 있음.

In [15]:
for k,v in episode.info['object_labels'].items():
    print(f"{k} is {v}")
    print(f"{v} is at {sim.get_rigid_object_manager().get_object_by_handle(k).translation}")

004_sugar_box_:0000 is any_targets|0
any_targets|0 is at Vector(-7.4937, 2.12802, -0.71232)
009_gelatin_box_:0000 is any_targets|1
any_targets|1 is at Vector(-13.1917, 0.47981, -7.28106)


### 5. Build plannner

pddl_problem.solution에 pddl solution 목록이 적혀있습니다.  
이걸 parsing해서 planning을 수행합니다.

In [16]:
for i, s in enumerate(pddl_problem.solution):
    print(f"step {i}:\n {s}")


step 0:
 <Action nav_to_goal ([obj-T:movable_entity_type, robot-T:robot_entity_type])->([any_targets|0-T:movable_entity_type, robot_0-T:robot_entity_type])>
step 1:
 <Action pick ([obj-T:movable_entity_type, robot-T:robot_entity_type])->([any_targets|0-T:movable_entity_type, robot_0-T:robot_entity_type])>
step 2:
 <Action nav_to_obj ([obj-T:goal_entity_type, robot-T:robot_entity_type])->([TARGET_any_targets|0-T:goal_entity_type, robot_0-T:robot_entity_type])>
step 3:
 <Action place ([place_obj-T:movable_entity_type, obj-T:goal_entity_type, robot-T:robot_entity_type])->([any_targets|0-T:movable_entity_type, TARGET_any_targets|0-T:goal_entity_type, robot_0-T:robot_entity_type])>
step 4:
 <Action nav_to_goal ([obj-T:movable_entity_type, robot-T:robot_entity_type])->([any_targets|1-T:movable_entity_type, robot_0-T:robot_entity_type])>
step 5:
 <Action pick ([obj-T:movable_entity_type, robot-T:robot_entity_type])->([any_targets|1-T:movable_entity_type, robot_0-T:robot_entity_type])>
step 6:

이렇게 pddl_problem의 각 stage goals이 적혀있고, 총 4가지의 sub goal이 있음을 확인할 수 있습니다.

In [17]:
pddl_problem.stage_goals

{'stage_1_1': (LogicalExprType.AND: [<Predicate: holding [[obj-T:movable_entity_type, robot-T:robot_entity_type]] [[any_targets|0-T:movable_entity_type, robot_0-T:robot_entity_type]]>],
 'stage_1_2': (LogicalExprType.AND: [<Predicate: object_at [[obj-T:movable_entity_type, at_entity-T:static_obj_type]] [[any_targets|0-T:movable_entity_type, TARGET_any_targets|0-T:goal_entity_type]]>],
 'stage_2_1': (LogicalExprType.AND: [<Predicate: holding [[obj-T:movable_entity_type, robot-T:robot_entity_type]] [[any_targets|1-T:movable_entity_type, robot_0-T:robot_entity_type]]>],
 'stage_2_2': (LogicalExprType.AND: [<Predicate: object_at [[obj-T:movable_entity_type, at_entity-T:static_obj_type]] [[any_targets|1-T:movable_entity_type, TARGET_any_targets|1-T:goal_entity_type]]>]}

pddl에는 각 stage_goals에 활용하는 entities에 순서를 매겨 저장합니다.  
oracle_nav는 이걸 활용해 navigation을 수행합니다.  

In [18]:
ordered_entities = pddl_problem.get_ordered_entities_list()

In [19]:
pddl_problem.get_ordered_entities_list()

[TARGET_any_targets|0-T:goal_entity_type,
 TARGET_any_targets|1-T:goal_entity_type,
 any_targets|0-T:movable_entity_type,
 any_targets|1-T:movable_entity_type,
 robot_0-T:robot_entity_type]

반면 oracle_pick은 object의 sim 속 id가 필요합니다.  
따라서, rigid_object_manager를 통해 가져옵니다.  

In [20]:
print(sim.get_rigid_object_manager().get_object_id_by_handle("002_master_chef_can_:0000"))


285


receptacle 역시 같은 방법으로 가져올 수 있습니다.  
하지만, 위에 올려 놓는 task이기 때문에, 위치를 조정해서 oracle_place에 전달합니다.  

In [21]:
import magnum as mn  # magnum 라이브러리 사용
print(sim.get_rigid_object_manager().get_object_by_handle("823e68a00d4aa37069b991bdf75f2d2cced124a3_:0000").translation)

old_position = mn.Vector3(-2.28042, 0.381002, -2.64413)  # 기존 위치
new_position = mn.Vector3(old_position.x, old_position.y + 0.5, old_position.z)  # Y 값 증가

print(new_position)  # Vector3(-2.28042, 0.481002, -2.64413)

Vector(-2.28042, 0.381002, -2.64413)
Vector(-2.28042, 0.881002, -2.64413)


parsing하는 함수를 만들어 놓고, episode마다 활용합니다.

In [22]:
# environment를 parsing해서 원하는 Object의 sim 속 id와 goal의 위치를 반환 (각각 pick & place 시 사용)
def parse_env(episode, sim): # obj_id for pick, goal_pos for place
     # 원하는 object의 id만
     obj_ids = {}
     for i, target in enumerate(episode.targets):
          print(target)
          obj_ids[i] = sim.get_rigid_object_manager().get_object_id_by_handle(target)
     
     import magnum as mn  # magnum 라이브러리 사용
     goal_poses = {}
     for i, goal_receptacle in enumerate(episode.goal_receptacles):
          old_position = sim.get_rigid_object_manager().get_object_by_handle(goal_receptacle[0]).translation
          goal_poses[i] = mn.Vector3(old_position.x, old_position.y + 0.3, old_position.z)  # Y 값 증가
          
     return obj_ids, goal_poses

### 6. Task 수행하기

In [24]:
import numpy as np
from habitat_sim.utils import viz_utils as vut
import re

habitat_env.reset()
habitat_env, episode, task, sim = make_setting(env)
pddl_problem, init, goal, target_ids, obj_ids = parse_pddl(task)
obj_ids, goal_poses = parse_env(episode, sim)

for key, value in episode.targets.items():
    print(f"target {key}: {sim.get_rigid_object_manager().get_object_by_handle(key).translation}")

for k, v in goal_poses.items():
    print(f"goal {k}: {v}")

def parse_action(pddl_problem, index):
    return pddl_problem.solution[index].name

def find_ordered_entity_idx(pattern, ordered_entities):
    for i, entity in enumerate(ordered_entities):
        if re.search(pattern, entity.name):
            return i
    return None


observations = []
for i, s in enumerate(pddl_problem.solution):
    should_end = False
    target_idx = int(s._param_values[0].name.split('|')[-1])
    cur_action = parse_action(pddl_problem, i)
    print("==============",cur_action, "==============")
    if cur_action == "nav_to_goal":
        ordered_entity_idx = find_ordered_entity_idx(rf"TARGET_any_targets\|{target_idx}", 
                                                     ordered_entities) + 1
    elif cur_action == "nav_to_obj":
        ordered_entity_idx = find_ordered_entity_idx(rf"^(?!TARGET_).*any_targets\|{target_idx}", 
                                                     ordered_entities) + 1
    
    while not should_end:
        if cur_action == "nav_to_obj" or cur_action == "nav_to_goal": # any_targets|target_idx로 이동
            action_dict = {
                "action": ("oracle_nav_action"),
                "action_args": {
                    "oracle_nav_action": np.array([ordered_entity_idx]) # 현재 target_ids 전달
                }
            }
            obs = habitat_env.step(action_dict)
            observations.append(obs)
            
            # if habitat_env.get_metrics()['pddl_subgoal_reward'] == 1:
            if obs['has_finished_oracle_nav']:
                print("has_finished_oracle_nav: ", obs['has_finished_oracle_nav'])
                print("agent_state: ", env.env.env._env.sim.get_agent_state())
                print("object_to_goal_distance: ", habitat_env.get_metrics()['object_to_goal_distance'])
                should_end = True
            
        elif cur_action == "pick":
            action_dict = {
                "action": ("oracle_pick"),
                "action_args": {
                    "oracle_grip_action": np.array([1], dtype=int),
                    "pick_action_object_idx": np.array([obj_ids[target_idx]], dtype=int)
                }
            }
            obs = habitat_env.step(action_dict)
            observations.append(obs)
            
            if obs['is_holding']:
                print("is_holding: ", obs['is_holding'])
                print("pddl_subgoal_reward: ", habitat_env.get_metrics()['pddl_subgoal_reward'])
                print("is grasping: ", env.env.env._env.sim.grasp_mgr.is_grasped)
                print("grasping obj: ", sim.get_rigid_object_manager().get_object_by_id(env.env.env._env.sim.grasp_mgr._snapped_obj_id).handle)
                
                should_end = True
            
        elif cur_action == "place":
            action_dict = {
                "action": ("oracle_place"),
                "action_args": {
                    "release_flag": np.array([1], dtype=int),
                    "target_position": goal_poses[target_idx]
                }
            }
            
            print("is grasping: ", env.env.env._env.sim.grasp_mgr.is_grasped)
            print("obs position: ", sim.get_rigid_object_manager().get_object_by_id(obj_ids[target_idx]).translation)
            obs = habitat_env.step(action_dict)
            observations.append(obs)
            
            if not obs['is_holding']:
                print("is_holding: ", obs['is_holding'])
                print("pddl_subgoal_reward: ", habitat_env.get_metrics()['pddl_subgoal_reward'])
                should_end = True
                print("obs position: ", sim.get_rigid_object_manager().get_object_by_id(obj_ids[target_idx]).translation)
            
    
print("==============end==============")
for k, v in habitat_env.get_metrics().items():
    print(k, v)


004_sugar_box_:0000
004_sugar_box_:0001
target 004_sugar_box_:0000: Vector(-1.87614, 1.00114, 2.30549)
target 004_sugar_box_:0001: Vector(-2.86778, 0.47588, -6.22352)
goal 0: Vector(-14.7651, 1.39583, 3.51743)
goal 1: Vector(-18.4455, 0.694001, -8.56654)
============== nav_to_goal ==============
has_finished_oracle_nav:  [1.]
agent_state:  AgentState(position=Vector(-12.4531, 0.162632, 2.97519), rotation=quaternion(0.555986881256104, 0, 0.831191062927246, 0))
object_to_goal_distance:  {'0': 12.412746, '1': 15.1194}
============== pick ==============
is_holding:  [1.]
pddl_subgoal_reward:  1.0
is grasping:  True
grasping obj:  004_sugar_box_:0000
============== nav_to_obj ==============
has_finished_oracle_nav:  [1.]
agent_state:  AgentState(position=Vector(-12.2247, 0.162632, 3.94789), rotation=quaternion(0.93721330165863, 0, 0.348756700754166, 0))
object_to_goal_distance:  {'0': 2.0271, '1': 15.1194}
============== place ==============
is grasping:  True
obs position:  Vector(-12.5212

In [25]:
vut.make_video(
    observations,
    "third_rgb",
    "color",
    "example_video",
    open_vid=True,
)

Encoding the video: example_video.mp4 


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (228, 240) to (240, 240) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x5571a7596b00] Warning: data is not aligned! This can lead to a speed loss
